In [1]:
import numpy as np
from astropy import units as u

from poliastro.bodies import Earth
from poliastro.twobody import propagation

In [2]:
k = Earth.k
r0 = [1131.340, -2282.343, 6672.423] * u.km
v0 = [-5.64305, 4.30333, 2.42879] * u.km / u.s
tof = 40 * u.min

In [3]:
r0

<Quantity [ 1131.34 ,-2282.343, 6672.423] km>

In [4]:
v0

<Quantity [-5.64305, 4.30333, 2.42879] km / s>

In [5]:
k

<Quantity 398600.0 km3 / s2>

In [6]:
tof

<Quantity 40.0 min>

In [7]:
propagation.kepler?

In [8]:
r, v = propagation.kepler(k.to(u.km**3 / u.s**2).value,
                          r0.to(u.km).value,
                          v0.to(u.km / u.s).value,
                          tof.to(u.s).value)
r *= u.km
v *= u.km / u.s

In [9]:
r

<Quantity [-4219.77617109, 4363.04569716,-3958.74972246] km>

In [10]:
v

<Quantity [ 3.68983773,-1.91670933,-6.11251847] km / s>

Prepare the inputs to measure performance:

In [11]:
k_ = k.to(u.km**3 / u.s**2).value
r0_ = r0.to(u.km).value
v0_ = v0.to(u.km / u.s).value
tof_ = tof.to(u.s).value

In [12]:
%timeit propagation.kepler(k_, r0_, v0_, tof_)

The slowest run took 4.96 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 9.81 µs per loop


Now we write our own implementation in pure Python. We will use numbified $c_2$ and $c_3$ functions for the moment.

In [13]:
from poliastro.stumpff import c2, c3

def kepler_py(k, r0, v0, tof):
    # Prepare input
    r0 = np.asarray(r0).astype(np.float)
    v0 = np.asarray(v0).astype(np.float)
    tof = float(tof)
    assert r0.shape == (3,)
    assert v0.shape == (3,)

    # Cache some results
    dot_r0v0 = np.dot(r0, v0)
    norm_r0 = np.linalg.norm(r0)
    sqrt_mu = np.sqrt(k)
    alpha = -np.dot(v0, v0) / k + 2 / norm_r0

    # Newton-Raphson iteration on the Kepler equation
    # Conservative initial guess
    xi = sqrt_mu * tof / norm_r0
    numiter = 50
    count = 0
    while count < numiter:
        psi = xi**2 * alpha
        norm_r = xi**2 * c2(psi) + dot_r0v0 / sqrt_mu * xi * (1 - psi * c3(psi)) + norm_r0 * (1 - psi * c2(psi))
        xi_new = xi + (sqrt_mu * tof - xi**3 * c3(psi) - dot_r0v0 / sqrt_mu * xi**2 * c2(psi) -
                       norm_r0 * xi * (1 - psi * c3(psi))) / norm_r
        err = np.abs((xi_new - xi) / xi_new)
        if err < 1e-10:
            break
        else:
            xi = xi_new
            count += 1
    else:
        print("No convergence")
        return

    # Compute Lagrange coefficients
    f = 1 - xi**2 / norm_r0 * c2(psi)
    g = tof - xi**3 / sqrt_mu * c3(psi)

    gdot = 1 - xi**2 / norm_r * c2(psi)
    fdot = sqrt_mu / (norm_r * norm_r0) * xi * (psi * c3(psi) - 1)

    # Return position and velocity vectors
    r = f * r0 + g * v0
    v = fdot * r0 + gdot * v0

    assert np.abs(f * gdot - fdot * g - 1) < 1e-6

    return r, v

In [14]:
r_py, v_py = kepler_py(k_, r0_, v0_, tof_)
r_py *= u.km
v_py *= u.km / u.s

In [19]:
np.testing.assert_array_almost_equal(r_py.value, r.value)
np.testing.assert_array_almost_equal(v_py.value, v.value)

In [20]:
r_py

<Quantity [-4219.77617109, 4363.04569716,-3958.74972246] km>

In [21]:
%timeit kepler_py(k_, r0_, v0_, tof_)

The slowest run took 4.76 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 70 µs per loop


In [22]:
%timeit propagation.kepler(k_, r0_, v0_, tof_)

The slowest run took 4.34 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 10.3 µs per loop


Our pure Python, non optimized version **is not *that* bad**! It stays in the same order of magnitude as the Fortran version for elliptic orbits at least. Let us see how can we improve the results.

In [23]:
%load_ext line_profiler

In [24]:
%lprun -f kepler_py kepler_py(k_, r0_, v0_, tof_)

According to line_profiler, 13 % of the time is spent in the two `asarray` calls and 14.5 % in the single `norm` computation.

In [25]:
%timeit np.linalg.norm(r0_)

The slowest run took 11.20 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 7.77 µs per loop


In [26]:
%timeit np.sqrt(np.dot(r0_, r0_))

The slowest run took 13.21 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.22 µs per loop


We can try and replace `linalg.norm` with the square root of the dot product. We can go further and assume the input are float arrays to save some more microseconds.

In [27]:
def kepler_py2(k, r0, v0, tof):
    # Algorithm parameters
    rtol = 1e-10
    numiter = 50
    
    # Cache some results
    dot_r0v0 = np.dot(r0, v0)
    norm_r0 = np.sqrt(np.dot(r0, r0))
    sqrt_mu = np.sqrt(k)
    alpha = -np.dot(v0, v0) / k + 2 / norm_r0

    # Newton-Raphson iteration on the Kepler equation
    # Conservative initial guess
    xi = sqrt_mu * tof / norm_r0
    count = 0
    while count < numiter:
        psi = xi * xi * alpha
        c2_psi = c2(psi)
        c3_psi = c3(psi)
        norm_r = xi * xi * c2_psi + dot_r0v0 / sqrt_mu * xi * (1 - psi * c3_psi) + norm_r0 * (1 - psi * c2_psi)
        xi_new = xi + (sqrt_mu * tof - xi * xi * xi * c3_psi - dot_r0v0 / sqrt_mu * xi * xi * c2_psi -
                       norm_r0 * xi * (1 - psi * c3_psi)) / norm_r
        if abs((xi_new - xi) / xi_new) < rtol:
            break
        else:
            xi = xi_new
            count += 1
    else:
        raise RuntimeError("Convergence could not be achieved under "
                           "%d iterations" % numiter)

    # Compute Lagrange coefficients
    f = 1 - xi**2 / norm_r0 * c2_psi
    g = tof - xi**3 / sqrt_mu * c3_psi

    gdot = 1 - xi**2 / norm_r * c2_psi
    fdot = sqrt_mu / (norm_r * norm_r0) * xi * (psi * c3_psi - 1)

    # Return position and velocity vectors
    r = f * r0 + g * v0
    v = fdot * r0 + gdot * v0

    assert np.abs(f * gdot - fdot * g - 1) < rtol

    return r, v

In [28]:
r_py2, v_py2 = kepler_py2(k_, r0_, v0_, tof_)
r_py2 *= u.km
v_py2 *= u.km / u.s

In [29]:
np.testing.assert_array_almost_equal(r_py2.value, r.value)
np.testing.assert_array_almost_equal(v_py2.value, v.value)

In [30]:
%timeit kepler_py2(k_, r0_, v0_, tof_)

The slowest run took 6.68 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 34.6 µs per loop


We cut the time in half! We still have another bullet: numba. This will require some work though, because we will have to provide some work arrays from outside to use the functions `np.abs` and `np.dot`, and in particular the latest one is not available.

In [31]:
import numba

def kepler_numba(k, r0, v0, tof, numiter=50, rtol=1e-10):
    """Propagates Keplerian orbit.

    Parameters
    ----------
    k : float
        Gravitational constant of main attractor (km^3 / s^2).
    r0 : array
        Initial position (km).
    v0 : array
        Initial velocity (km).
    tof : float
        Time of flight (s).
    numiter : int, optional
        Maximum number of iterations, default to 50.
    rtol : float, optional
        Maximum relative error permitted, default to 1e-10.

    Raises
    ------
    RuntimeError
        If the algorithm didn't converge.

    Notes
    -----
    This algorithm is based on Vallado implementation, and does basic Newton
    iteration on the Kepler equation written using universal variables. Battin
    claims his algorithm uses the same amount of memory but is between 40 %
    and 85 % faster.

    """
    # Compute Lagrange coefficients
    try:
        f, g, fdot, gdot = _kepler(k, r0, v0, tof, numiter, rtol)
    except RuntimeError:
        raise RuntimeError("Convergence could not be achieved under "
                           "%d iterations" % numiter)

    assert np.abs(f * gdot - fdot * g - 1) < rtol

    # Return position and velocity vectors
    r = f * r0 + g * v0
    v = fdot * r0 + gdot * v0

    return r, v

@numba.njit('f8(f8[:], f8[:])')
def dot(u, v):
    dp = 0.0
    for ii in range(u.shape[0]):
        dp += u[ii] * v[ii]
    return dp

@numba.njit
def _kepler(k, r0, v0, tof, numiter, rtol):
    # Cache some results
    dot_r0v0 = dot(r0, v0)
    norm_r0 = dot(r0, r0) ** .5
    sqrt_mu = k**.5
    alpha = -dot(v0, v0) / k + 2 / norm_r0

    # Newton-Raphson iteration on the Kepler equation
    # Conservative initial guess
    xi = sqrt_mu * tof / norm_r0
    xi_new = xi
    count = 0
    while count < numiter:
        psi = xi * xi * alpha
        c2_psi = c2(psi)
        c3_psi = c3(psi)
        norm_r = xi * xi * c2_psi + dot_r0v0 / sqrt_mu * xi * (1 - psi * c3_psi) + norm_r0 * (1 - psi * c2_psi)
        xi_new = xi + (sqrt_mu * tof - xi * xi * xi * c3_psi - dot_r0v0 / sqrt_mu * xi * xi * c2_psi -
                       norm_r0 * xi * (1 - psi * c3_psi)) / norm_r
        if abs((xi_new - xi) / xi_new) < rtol:
            break
        else:
            xi = xi_new
            count += 1
    else:
        raise RuntimeError

    # Compute Lagrange coefficients
    f = 1 - xi**2 / norm_r0 * c2_psi
    g = tof - xi**3 / sqrt_mu * c3_psi

    gdot = 1 - xi**2 / norm_r * c2_psi
    fdot = sqrt_mu / (norm_r * norm_r0) * xi * (psi * c3_psi - 1)

    return f, g, fdot, gdot

In [32]:
r_numba, v_numba = kepler_numba(k_, r0_, v0_, tof_)
r_numba *= u.km
v_numba *= u.km / u.s

In [33]:
np.testing.assert_array_almost_equal(r_numba.value, r.value)
np.testing.assert_array_almost_equal(v_numba.value, v.value)

In [34]:
r_numba

<Quantity [-4219.77617109, 4363.04569716,-3958.74972246] km>

In [35]:
v_numba

<Quantity [ 3.68983773,-1.91670933,-6.11251847] km / s>

In [36]:
%timeit kepler_numba(k_, r0_, v0_, tof_)

The slowest run took 7.97 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 11.7 µs per loop


**We achieved a 0.9x factor with the pure Python version**. We still have to test what happens for longer times and for non-elliptic orbits and compare both cases, but in any case the results are promising.

Another interesting side-thought: can I use the `scipy.optimize` functions with numbified callbacks? Does it save performance in this case?